In [1]:
import os
import subprocess
import uuid
import asyncio
import yaml
import chromadb
import requests
import base64
import zlib
import json
import argparse
import tempfile
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_core.models import ChatCompletionClient, ModelInfo
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination, TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.memory.canvas import TextCanvasMemory
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, UnstructuredWordDocumentLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
AZURE = "/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/src/model_config_azure.yaml"
OLLAMA = "/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/src/model_config_ollama.yaml"
LMSTUDIO = "/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/src/model_config_llmstudio.yaml"

# Load model configuration and create the model client.
with open(LMSTUDIO, "r") as f:
    model_config = yaml.safe_load(f)
    
model_client = ChatCompletionClient.load_component(model_config)

Agent to make diagram

In [3]:
def read_txt_file(file_path: str) -> str:
    """
    Reads the content of a .txt file and returns it as a string.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"
    
def read_pdf_file(file_path: str) -> str:
    """
    Reads the content of a .pdf file and returns it as a string.
    """
    try:
        loader = PyMuPDFLoader(file_path)
        documents = loader.load()
        return documents[0].page_content if documents else ""
    except Exception as e:
        return f"Error reading file: {e}"
    
def write_mermaid_to_file(mermaid_code: str, filename: str) -> None:
    """
    Write Mermaid diagram code to a .mmd file.

    Args:
        mermaid_code (str): The Mermaid syntax/code to write.
        filename (str): The name of the file to write to. Should end with '.mmd'.
    """
    if not filename.endswith('.mmd'):
        raise ValueError("Filename must end with '.mmd'")
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("```mermaid\n")
        file.write(mermaid_code)
        file.write("\n```")

    print(f"Mermaid diagram written to {filename}")


In [4]:
 # --- Set up the Canvas-based Memory ---
text_canvas_memory = TextCanvasMemory()  # Canvas is created internally

    # --- Tools for the Writer to create/update files on the canvas ---
update_file_tool = text_canvas_memory.get_update_file_tool()


In [5]:
planning_agent = AssistantAgent(
    "planning_agent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    handoffs=["summarizer_agent", "diagram_creator_agent"],
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
    - summarizer_agent: An agent reads pdf files and summarizes the process information.
    - diagram_creator_agent: An agent that receives the summary of a process from the summarizer Agent and creates diagrams as a code using Mermaid languagge.
    - mermaid_code_reviewer_agent: An agent that reviews the Mermaid code and provides feedback to the diagram_creator_agent.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After assigning tasks, wait for all agents to finish their tasks.
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
    model_client_stream=True,
)

summarizer_agent = AssistantAgent(
        name="summarizer_agent",
        description="An agent that reads pdf files and summarizes the information.",
        model_client=model_client,
        handoffs=["diagram_creator_agent"],
        tools=[read_pdf_file, update_file_tool],
        system_message=(
            """You are an expert on making process summaries."
            You will read a pdf file using tools and summarize the information.
            Make sure to include all the important information of the process.
            The summary should be in the format: 'Summary: <summary>'
            Use update_file_tool to store it in a file named 'Sales_SOP_acquiring_clients.md' on the canvas memory.
            When the summary is complete respond: 'Summary complete. Handing off to diagram_creator_agent.'"""
        ),
        memory=[text_canvas_memory],
        model_client_stream=True,  # Enable model client streaming.
    )

diagram_creator_agent = AssistantAgent(
    name="diagram_creator_agent",
    description="An agent that create diagrams as a code.",
    model_client=model_client,
    handoffs=["mermaid_code_reviewer_agent"],
    tools=[update_file_tool],
    system_message="""
    You are a Process Diagram Creator and an expert in the Mermaid language.
    Your job is to create process diagrams as a code using the Mermaid language from the 'Sales_SOP_acquiring_clients.md' in the canvas.
    Store the generated Mermaid code with update_file_tool on the canvas in a file named "Sales_SOP_acquiring_clients_dia.md then hand off to the mermaid_code_reviewer_agent."
    If the mermaid_code_reviewer_agent provides feedback, you will update the code with update_file_tool in the canvas with the changes then hand off to the mermaid_code_reviewer_agent.
    When the diagram code is complete, respond with "Diagram code complete. Handing off to mermaid_code_reviewer_agent."
    """,
    memory=[text_canvas_memory],
    model_client_stream=True,
)

mermaid_code_reviewer_agent = AssistantAgent(
        name="mermaid_code_reviewer_agent",
        description="An agent that reviews the Mermaid code.",
        model_client=model_client,
        handoffs=["diagram_creator_agent"],
        tools=[write_mermaid_to_file],
        system_message=(
            """You are an expert on the Mermaid languagge.
            Review the Diagram Mermaid code stored in 'Sales_SOP_acquiring_clients_dia.md' on the canvas.
            If there is feedback, respond with the feedback.
            if there is no feedback, writte the code with write_mermaid_to_file to '/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/diagram.mmd'
            and respond with 'Diagram code complete you can TERMINATE. Handing off to the PlanningAgent.'"""
        ),
        memory=[text_canvas_memory],
)

In [6]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [17]:
selector_prompt = """Select an agent to perform task, based on teh roles and the current conversation context.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [8]:
team = SelectorGroupChat(
    [planning_agent, summarizer_agent, diagram_creator_agent, mermaid_code_reviewer_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [7]:
team = RoundRobinGroupChat([planning_agent, summarizer_agent, diagram_creator_agent, mermaid_code_reviewer_agent], termination_condition=termination)


In [7]:
team = Swarm(
    [planning_agent, summarizer_agent, diagram_creator_agent, mermaid_code_reviewer_agent],
    termination_condition=termination,
)

In [8]:
task = "Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language"

In [9]:
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language
---------- ModelClientStreamingChunkEvent (planning_agent) ----------
1. summarizer_agent : Read the PDF located at '/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' and summarize the sales process described within it.

---------- ToolCallRequestEvent (planning_agent) ----------
[FunctionCall(id='351334686', arguments='{}', name='transfer_to_summarizer_agent')]
---------- ToolCallExecutionEvent (planning_agent) ----------
[FunctionExecutionResult(content='Transferred to summarizer_agent, adopting the role of summarizer_agent immediately.', name='transfer_to_summarizer_agent', call_id='351334686', is_error=False)]
---------- HandoffMessage (planning_agent) ----------
Transferred to summarizer_agent, adopting the ro

/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/venv/lib/python3.13/site-packages/autogen_core/_routed_agent.py:268: UserWarning: No text content or tool calls are available. Model returned empty result.
  return_value = await func(self, message, ctx)  # type: ignore


---------- TextMessage (summarizer_agent) ----------

---------- MemoryQueryEvent (summarizer_agent) ----------
[MemoryContent(content='=== CANVAS FILES ===\nFile: Sales_SOP_acquiring_clients.md (rev 1):\n# Sales SOP for acquiring new clients\n\nThe Sales SOP for acquiring new clients outlines a straightforward process for business and sales development professionals.\n\nIt involves creating a list of potential clients, finding their email addresses, sending introductory emails highlighting product benefits, scheduling phone calls upon reply, completing those conversations, and forwarding interested client information to the sales manager.\n\nSupporting resources include an email template, phone conversation guidelines, and access to potential client lists/databases.\n\n=== END OF CANVAS ===', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- TextMessage (summarizer_agent) ----------

---------- MemoryQueryEvent (summarizer_agent) ----------
[MemoryContent(conte

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language", type='TextMessage'), ThoughtEvent(source='planning_agent', models_usage=None, metadata={}, content="1. summarizer_agent : Read the PDF located at '/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' and summarize the sales process described within it.\n", type='ThoughtEvent'), ToolCallRequestEvent(source='planning_agent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='351334686', arguments='{}', name='transfer_to_summarizer_agent')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='planning_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Transferred to summarizer_age